# Normalized ISBT and Stratigraphic Column Generator

## Summary
The below kernal process data that has been cleaned using the "CPT_New" module. Raw CPT Data is assigned to variables and used to calculate a normalized Robertson Soil Behavior Type, then plots the results in a three plot figure showing (1) the raw CPT resistance and sleeve:tip ratio, (2) the Ic classification against soil blocks, and (2) a stratigraphic columns plotting the SBT. Special consideration is given to soil with high rf and fs as this has been attributed to glauconitic soils. If encountered, the model will created a SBT block based on the measured soil type, but highlights the block bright red. The soil in the highlighted intervals should be considered suspect glauconite-baring soil.

This module has only been QC'd for CPT test conducted on the seabed, below sea level

https://www.rocscience.com/help/settle/pdf_files/theory/CPT_Theory_Manual.pdf

## Data Requirements 

CPT Test Depth;
CPT Test Number;
Tip Resistance (MPa);
Sleeve resistance (MPa);
Pore Pressure (MPa);

In [2]:
# Import modules =========================================================================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib import pyplot as plt
from matplotlib.lines import Line2D

# Input file =============================================================================
class CPT:
    def __init__(self,filename,gwd):
        self.filename = filename
        self.gwd = gwd
        
        #self._load_file_data()
        
    def ISBT(self):
        self.df = pd.read_csv(self.filename)
        #self.df = self.dfu.dropna(how='any', subset=['SCPT_FRES'])
        ndata = self.df.shape[0]
        
    # Raw CPT Data from dataset============================================================
        t = self.df['SCPG_TESN']                                # CPT test number
        z = self.df['SCPT_DPTH']                                # Depth
        qc = self.df['SCPT_RES']                                # Cone Resistance in MPa
        fs = self.df['SCPT_FRES']                               # Sleeve Friction in MPa
        u2 = self.df['SCPT_PWP2']                               # Us pore pressure
    
    #def calculate_ISBT(self):
    # Calculated Values=====================================================================
        rf = (fs/qc)*100                                        # fs/qc ratio
        dz = z[1]-z[0]                                          # Change of depth
        wl = self.gwd                                           # Water depth in meters
        h = abs(wl)                                             # Height of water column
        soil_den = 18                                           # Soil density
        max_depth = 40                                          # Chart display depth meters
        rho = 1023.6                                            # Density of salt water kg/m^3
        P = round((1023.6*9.80665*h)/1000000,4)                 # Hydrostatic water pressure MPa
    # Vertical Stress ======================================================================
        gamma = [10*(0.27*np.log10(x)+0.36*np.log10(y/P)+1.236) for x,y in zip(fs,qc)]# Soil Unit Weight
        u0 = wl + z * 10                                        # Hydrostatic water pressure on cone
        udl = (u2-u0)
        sig_tot = z*18                                          # Total stress
        sig_eff = sig_tot - u0                                  # Effective stress
        qt = (qc*1000) + 0.21+u2                                # qt in KPa
        qn = (qt - sig_tot)                                     # Net cone resistance
        Bq = (udl/(qn))                                         # Pore pressure ratio
    # I-SBT (Robertson)=====================================================================
        Qt = (qt - sig_tot)/sig_eff                             # Normalized cone resistance
        Fr = ((fs*1000)/(qt-sig_eff))*100                       # Normalized frisction ratio
        Ic = [np.sqrt((3.47-np.log10(x))**2 + (np.log10(y)+1.22)**2) for x,y in zip (Qt,Fr)]
        
        index = []
        for i in range(ndata):
            if Ic[i] > 3.6:
                index.append(2)
            elif Ic[i] > 2.95 and Ic[i] <=3.6:
                index.append(3)
            elif Ic[i] > 2.60 and Ic[i] <=2.95:
                index.append(4)
            elif Ic[i] > 2.05 and Ic[i] <=2.60:
                index.append(5)
            elif Ic[i] > 1.31 and Ic[i] <=2.05:
                index.append(6)
            else:
                index.append(7)
                
        glauc = [] # Default is fs=0.45 and rf=3.5
        for i in range(ndata):
            if fs[i] > 0.4 and rf[i] > 3.0: 
                glauc.append(6)
            else:
                glauc.append(0)
                
    # Hydraulic Conductivity ==============================================================================
    #def calculate_K():
        K = []
        for i in range(ndata):
            if Ic[i] <= 3.27:
                K.append(10**(0.952-3.04*Ic[i]))
            else:
                K.append(10**(-4.52-1.37*Ic[i]))
                
# PLOTTING =================================================================================================
        my_df = pd.DataFrame(list(zip(t,z,qc,u2,fs,rf,K,Ic )), 
               columns =['Test','Depth', 'Tip','Sleeve','Pore Pressure','Fric Ratio','K','SBT']) 
        
        custom_lines = [Line2D([0], [0], color='green', lw=2),
                        Line2D([0], [0], color='black', lw=2)]
    #def plot_ISBT(self):
        ylim = 40.0
        fig = plt.figure(figsize=(10,11))          
        
        ax1 = fig.add_subplot(131)
        ax2 = ax1.twiny()
        ax3 = fig.add_subplot(132)
        
        g = my_df.groupby('Test')
        for grp, indices in g.groups.items():
            df_temp = my_df.loc[indices]
                    
            ax1.plot(df_temp['Tip'],df_temp['Depth'], color='black', label="qc")
            ax1.plot([100,100],[0,10],color='green', label="Rf") # this is only dummy
            ax1.set_ylim(0,ylim)
            ax1.set_xlim(0,100)
            
            ax2.plot(df_temp['Fric Ratio'],df_temp['Depth'],color='green', label="Rf")
            ax3.plot(df_temp['SBT'],df_temp['Depth'],color='yellow',linewidth=2)
            
            ax1.minorticks_on()
            ax1.grid(True, which='both')
            ax1.legend(custom_lines,['Rf','qc'],loc=3)

            ax1.set_ylabel("depth (m)",size=12)
            ax1.set_xlabel("qc (MPa)",size=12)                    
            
            ax2.set_ylim(0,ylim)
            ax2.set_xlim(0,25.0)
            ax2.invert_xaxis()
            ax2.invert_yaxis()
            ax2.set_xlabel("Rf (%)",size=12)
        
        ax1.invert_yaxis() # inversion is overrided if scripted before ax2 code
           
        ax3.set_ylim(0,ylim)
        ax3.set_xlim(1.0,4.0)
        ax3.invert_yaxis()
        ax3.add_patch(patches.Rectangle((0,0),1.31,ylim,facecolor='darkgoldenrod'))
        ax3.add_patch(patches.Rectangle((1.31,0),0.74,ylim,facecolor='goldenrod'))
        ax3.add_patch(patches.Rectangle((2.05,0),0.55,ylim,facecolor='mediumseagreen'))
        ax3.add_patch(patches.Rectangle((2.60,0),0.35,ylim,facecolor='seagreen'))
        ax3.add_patch(patches.Rectangle((2.95,0),0.65,ylim,facecolor='cadetblue'))
        ax3.add_patch(patches.Rectangle((3.6,0),0.4,ylim,facecolor='sienna'))
        ax3.yaxis.grid(which="minor")
        ax3.yaxis.grid(which="major")
        ax3.xaxis.grid(which="major")
        ax3.minorticks_on()
        ax3.text(1.05, 25, "gravel - dense sand", va='bottom', rotation=90, size=13, color="black")
        ax3.text(1.55, 25, "clean sand - silty sand", va='bottom', rotation=90, size=13, color="black")
        ax3.text(2.22, 25, "silty sand - sandy silt", va='bottom', rotation=90, size=13, color="black")
        ax3.text(2.65, 25, "clayey silt - silty clay", va='bottom', rotation=90, size=13, color="black")
        ax3.text(3.2, 25, "silty clay - clay", va='bottom', rotation=90, size=13, color="black")
        ax3.text(3.65, 25, "peat", va='bottom', rotation=90, size=13, color="black")
        ax3.set_xlabel("Ic",size=12)
        split_name = self.filename.split('.')
        ax3.set_title(split_name[0]) 
        
        ax4 = fig.add_subplot(133)
        for i in range(ndata-1):
            if glauc[i+1] == 6:
                colour = 'red'
            elif index[i+1] == 2:
                colour = 'sienna'
            elif index[i+1] == 3:
                colour = 'cadetblue'
            elif index[i+1] == 4:
                colour = 'seagreen'
            elif index[i+1] == 5:
                colour = 'mediumseagreen'
            elif index[i+1] == 6:
                colour = 'goldenrod'
            elif index[i+1] == 7:
                colour = 'darkgoldenrod'
            ax4.add_patch(patches.Rectangle((0,z[i]),index[i+1],(0.1),facecolor=colour))
            ax4.add_patch(patches.Rectangle((0,z[i]),index[i+1],(0.1),facecolor=colour))   
        ax4.set_ylim(0,ylim)
        ax4.set_xlim(0,10.0)
        ax4.invert_yaxis()
        ax4.yaxis.grid(which="major")
        ax4.yaxis.grid(which="minor")
        ax4.xaxis.grid(which="major")
        ax4.minorticks_on()
        
        # Save figure =========================================================
        d2 = "C:/Users/schoenmannst/Documents/01 PROJECTS/2020 Vineyard Wind/04-SBT_StratCol_Figures"
        plt.savefig(d2+"/"+split_name[0]+".png", facecolor=fig.get_facecolor(), transparent=False)
        
        plt.close(fig)      # Closes figure during iteration to save RAM. Block out if running module on individual location.


## Run Individual Location

Used to run module on a single soil boring location

In [3]:
# Result
C1 = CPT('20T026.ags',-45) # filename, ground water depth in meters
C1.ISBT()

C:\Users\schoenmannst\AppData\Local\Continuum\miniconda3\lib\site-packages\ipykernel_launcher.py:51: RuntimeWarning: invalid value encountered in log10


## Iterate Accross Multiple CPT Locations 

Iterate module through a specific directory containing all CPT data

In [4]:
import os 
directory = "C:/Users/schoenmannst/Documents/01 PROJECTS/2020 Vineyard Wind/03-Cleaned_CPT_Data"
d2 = "C:/Users/schoenmannst/Documents/01 PROJECTS/2020 Vineyard Wind/04-SBT_StratCol_Figures"
    
for filename in os.listdir(directory):
    if filename.endswith(".ags"): 
        C1 = CPT(filename,-45) # filename, ground water depth in meter
        C1.ISBT()
        continue

C:\Users\schoenmannst\AppData\Local\Continuum\miniconda3\lib\site-packages\ipykernel_launcher.py:51: RuntimeWarning: invalid value encountered in log10
